In [3]:
import os
import shutil

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import numpy as np
import pydicom
import cv2

from tqdm import tqdm
from skimage.exposure import rescale_intensity
from PIL import Image
from duke_dbt_data import dcmread_image
from typing import AnyStr, BinaryIO, Dict, List, NamedTuple, Optional, Union

def npArrayNormalized(array,tamano_cuadrado):
    array_uint8 = array.astype(np.uint16)
    array_byte0 = (array_uint8 & 0xFF).astype(np.uint8)
    array_byte1 = ((array_uint8 >> 8) & 0xFF).astype(np.uint8)
    array_byte01 = []
    array_byte11 = []
    
    for i in range(array.shape[0]):  # Recorrer la dimensión 0 del array
        # Redimensionar y agregar los bytes a las listas
        array_byte01.append(cv2.resize(array_byte0[i], (tamano_cuadrado, tamano_cuadrado)))
        array_byte11.append(cv2.resize(array_byte1[i], (tamano_cuadrado, tamano_cuadrado)))
    
    # Convertir las listas de bytes a arrays de NumPy
    array_byte01 = np.array(array_byte01)
    array_byte11 = np.array(array_byte11)
    
    arrays = [array_byte01, array_byte11]
    
    arrays = np.transpose(np.array(arrays), (1, 2, 3, 0))
    return arrays
    
def resizeAndSave(npArrayImag, tamano_cuadrado, ruta):
    """
    Redimensiona cada imagen en un np.ndarray y las guarda en un archivo usando np.save.

    Args:
    - npArrayImag (np.ndarray): Un array de NumPy que contiene las imágenes.
    - tamano_cuadrado (int): El tamaño al que se redimensionarán las imágenes (ancho y alto).
    - ruta (str): La ruta donde se guardará el archivo con las imágenes redimensionadas.
    """
    # Crear un nuevo array para almacenar las imágenes redimensionadas
    imagenes_redimensionadas = np.zeros((5, tamano_cuadrado, tamano_cuadrado))

    # Redimensionar cada imagen y guardarla en el nuevo array
    for i in range(5):
        imagen_redimensionada = cv2.resize(npArrayImag[i], (tamano_cuadrado, tamano_cuadrado))
        imgu8 = convert(imagen_redimensionada, 0, 255, np.uint8)
        imagenes_redimensionadas[i] = imgu8

    # Guardar el array de imágenes redimensionadas en un archivo
    np.save(ruta, imagenes_redimensionadas)

In [5]:
# Leer los archivos CSV
df1 = pd.read_csv("D:/TFG/Codigo/InfValidation/file-paths-validation.csv")
df2 = pd.read_csv("D:/TFG/Codigo/InfValidation/labels-validation.csv")

# Fusionar los DataFrames en uno solo basado en las columnas comunes
merged_df = pd.merge(df1, df2, on=["PatientID", "StudyUID", "View"])
# Eliminar la columna "classic_path"
conteo_Normal = merged_df["Normal"].value_counts()
conteo_Actionable = merged_df["Actionable"].value_counts()
conteo_Benign = merged_df["Benign"].value_counts()
conteo_Cancer = merged_df["Cancer"].value_counts()

# Mostrar los distintos valores y sus conteos
print(conteo_Normal)
print(conteo_Actionable)
print(conteo_Benign)
print(conteo_Cancer)
merged_df = merged_df.drop(columns=["classic_path"])

# Mostrar las primeras filas del DataFrame fusionado para verificar la fusión
merged_df.head()

Normal
1    928
0    235
Name: count, dtype: int64
Actionable
0    1003
1     160
Name: count, dtype: int64
Benign
0    1125
1      38
Name: count, dtype: int64
Cancer
0    1126
1      37
Name: count, dtype: int64


,PatientID,StudyUID,View,descriptive_path,Normal,Actionable,Benign,Cancer
0,DBT-P00002,DBT-S03345,lcc,Breast-Cancer-Screening-DBT/DBT-P00002/01-01-2...,1,0,0,0
1,DBT-P00002,DBT-S03345,lmlo,Breast-Cancer-Screening-DBT/DBT-P00002/01-01-2...,1,0,0,0
2,DBT-P00002,DBT-S03345,rcc,Breast-Cancer-Screening-DBT/DBT-P00002/01-01-2...,1,0,0,0
3,DBT-P00002,DBT-S03345,rmlo,Breast-Cancer-Screening-DBT/DBT-P00002/01-01-2...,1,0,0,0
4,DBT-P00114,DBT-S03767,rmlo,Breast-Cancer-Screening-DBT/DBT-P00114/01-01-2...,0,0,0,1


In [3]:
# Guardar el DataFrame en un archivo pickle serializado
merged_df.to_pickle("data/Training/merged_dfTraining_serialized.pickle")

print(f"DataFrame guardado en data/merged_df_serialized.pickle")

DataFrame guardado en data/merged_df_serialized.pickle


In [4]:
# Cargar el DataFrame desde un archivo pickle serializado
merged_df = pd.read_pickle("data/Training/merged_dfTraining_serialized.pickle")

# Ahora puedes usar merged_df_recuperado como lo harías con cualquier DataFrame de Pandas
merged_df.head()

,PatientID,StudyUID,View,descriptive_path,Normal,Actionable,Benign,Cancer
0,DBT-P00013,DBT-S00163,rmlo,Breast-Cancer-Screening-DBT/DBT-P00013/01-01-2...,0,0,1,0
1,DBT-P00023,DBT-S04378,lcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,0,1,0,0
2,DBT-P00023,DBT-S04378,lmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,0,1,0,0
3,DBT-P00023,DBT-S04378,rcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,0,1,0,0
4,DBT-P00023,DBT-S04378,rmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,0,1,0,0


In [5]:
conteo_Normal = merged_df["Normal"].value_counts()
conteo_Actionable = merged_df["Actionable"].value_counts()
conteo_Benign = merged_df["Benign"].value_counts()
conteo_Cancer = merged_df["Cancer"].value_counts()

# Mostrar los distintos valores y sus conteos
print(conteo_Normal)
print(conteo_Actionable)
print(conteo_Benign)
print(conteo_Cancer)

Normal
1    18232
0      916
Name: count, dtype: int64
Actionable
0    18432
1      716
Name: count, dtype: int64
Benign
0    19024
1      124
Name: count, dtype: int64
Cancer
0    19072
1       76
Name: count, dtype: int64


# Faltantes
80?  


In [ ]:
# Tamaño cuadrado deseado para las imágenes (en píxeles)
tamano_cuadrado = 720

# Iterar sobre las filas del DataFrame merged_df
for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Cargando imágen"):
    if index>79:
        # Cargar la imagen .dcm desde el archivo
        # print(row["descriptive_path"])
        ruta_imagen = os.path.join("/media/miguel/SYSTEM/breastCancerlow/Bruto/manifest-1617905855234/", row["descriptive_path"])
    
        # Reemplazar "NA" con otro valor
        ruta_imagen = ruta_imagen.replace("000000-", "000000-NA-")
        
        if os.path.exists(ruta_imagen):
            view = row["View"]
            nombre = f"{view}-{index}.npy"
            imagen = dcmread_image(fp=ruta_imagen, view=view)
        
            # Cojemos 5 imagenes del total
            n = len(imagen)
            indices_seleccionados = [2, n // 4, n // 2, 3 * n // 4, n - 3]
            # Seleccionar las imágenes en las posiciones especificadas
            imagenes_seleccionadas = imagen[indices_seleccionados]
        
            arrayImagenes = npArrayNormalized(imagenes_seleccionadas,tamano_cuadrado)# Resize
            
            # Determinar a qué colección agregar la imagen según los valores de las columnas
            if row["Normal"] == 1:
                np.save(os.path.join("/media/miguel/SYSTEM/breastCancerlow/Bruto/Training/Normal", nombre), arrayImagenes)
                #print(os.path.join("imagenes/Normal", nombre))
            if row["Actionable"] == 1:
                np.save(os.path.join("/media/miguel/SYSTEM/breastCancerlow/Bruto/Training/Actionable", nombre), arrayImagenes)
                #print(os.path.join("imagenes/Actionable", nombre))
            if row["Benign"] == 1:
                np.save(os.path.join("/media/miguel/SYSTEM/breastCancerlow/Bruto/Training/Benign", nombre), arrayImagenes)
                #print(os.path.join("imagenes/Benign", nombre))
            if row["Cancer"] == 1:
                np.save(os.path.join("/media/miguel/SYSTEM/breastCancerlow/Bruto/Training/Cancer", nombre), arrayImagenes)
            #print(os.path.join("imagenes/Cancer", nombre))

Cargando imágen:   0%|                                | 0/19148 [00:00<?, ?it/s]/home/miguel/anaconda3/envs/deep/lib/python3.12/site-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
Cargando imágen:   0%|                   | 95/19148 [07:54<138:51:56, 26.24s/it]